In [43]:
from ultralytics import YOLO
from ultralytics.utils.metrics import mask_iou
from torch import bitwise_or, zeros, Tensor
import cv2
import numpy as np
import pandas as pd

model = YOLO('runs/segment/train3/weights/best.pt')

In [44]:
def read_and_normalize_mask(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(image, 0.5, 1, cv2.THRESH_BINARY)

    mask = mask.reshape(-1, int(image.shape[1] * image.shape[0]))

    return Tensor(mask)

def get_iou(yolo_model, img_path):
    results = yolo_model(img_path)
    true_mask = read_and_normalize_mask(img_path.replace("images", "masks"))
    
    assert len(results) == 1, "support only len == 1"
    for r in results:
        if isinstance(r.masks, type(None)): return None
        masks = r.masks.data.reshape(-1, 256*256).cpu()
        masks = np.any(np.array(masks), axis=0)
        masks = masks.reshape(-1, 256*256)
        print(masks)
        masks = Tensor(masks)
    
        return mask_iou(true_mask, masks)


In [45]:
%%capture

# get an average iou
import os
import numpy as np

image_directory = "./data/test(region_growth)/images/"

iou_values = []
for image_path in os.listdir(image_directory):
    if image_path.endswith(".png"):
        result = get_iou(model, os.path.join(image_directory, image_path))
        if result != None:
            iou = result.tolist()[0][0]
            iou_values.append(iou)

average_iou = np.mean(iou_values)
median = np.median(iou_values)


image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2016_08_13__23_00_05_84__SDO_AIA_AIA_193.png: 256x256 4 coronal holes, 28.3ms
Speed: 4.3ms preprocess, 28.3ms inference, 47.5ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2015_03_11__22_59_42_84__SDO_AIA_AIA_193.png: 256x256 3 coronal holes, 17.6ms
Speed: 1.2ms preprocess, 17.6ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2013_10_17__22_59_42_84__SDO_AIA_AIA_193_r180.png: 256x256 1 coronal hole, 20.7ms
Speed: 0.7ms preprocess, 20.7ms inference, 2.7ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images/2014_01_26__23_00_06_84__SDO_AIA_AIA_193_r270.png: 256x256 (no detections), 15.0ms
Speed: 0.7ms preprocess, 15.0ms inference, 0.7ms postproces

In [46]:
%%capture cell
val_res = model.val(data='./bakalarka-yolo.yaml', imgsz=256, split="test")

Ultralytics YOLOv8.0.203 🚀 Python-3.8.5 torch-2.1.0+cu121 CUDA:0 (NVIDIA TITAN RTX, 24210MiB)
val: New cache created: /home/deeplearningtitan/Studenti/tests/data/test(region_growth)/images-color.cache
                   all       2466       3505      0.766      0.445      0.505      0.376      0.776      0.434      0.496      0.309
Speed: 0.1ms preprocess, 4.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/segment/val14


In [48]:
dice = val_res.seg.f1
print("Dice score:", dice[0])
print("IoU:", average_iou)

#  results for colorful images
# Dice score: 0.5807742979901169
# IoU: 0.7006662135058757

# results for gray images
# Dice score: 0.5563138232924477
# IoU: 0.6986303846845785

Dice score: 0.5563138232924477
IoU: 0.6986303846845785
